<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Generative Adversarial Network</h2>

Based in [this post](https://towardsdatascience.com/an-easy-introduction-to-generative-adversarial-networks-6f8498dc4bcd)

In [1]:
import os

import torch
from torch import nn, optim
from torch.autograd.variable import Variable

import torchvision
import torchvision.transforms as transforms

In [ ]:
print(torch.__version__)

In [2]:
#torch.manual_seed(args.seed)

if torch.cuda.is_available():
    print("Using CUDA")
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Preprocessing
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Training data
train_set = torchvision.datasets.MNIST(root=os.path.join('.','..','data'),
                                      train=True,
                                      download=True,
                                      transform=transform)

train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=32,
                                          shuffle=True)

# Labels
classes = [str(i) for i in range(0,10)]

In [4]:
# Our Discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out.to(device)
        
discriminator = Discriminator().to(device)

In [5]:
# Our Generator class
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(x.size(0), 100)
        out = self.model(x) #.cuda()
        return out

generator = Generator().to(device)

In [6]:
# If we have a GPU with CUDA, use it
if torch.cuda.is_available():
    print("Using CUDA")
    discriminator.cuda()
    generator.cuda()

# Setup loss function and optimizers
lr = 0.0001
num_epochs = 40
num_batches = len(train_loader)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

1 -  We start by preparing our *real* image data for the discriminator. The input is a batch of real MNIST images. The output is a vector of all ones, since a 1 indicates that the image is real.

2 - Next, we’ll prepare our input vector for the generator such that we can generate fake images. Recall that our generator network takes an input vector of length 100, so that’s what we create here. The images.size(0) is for the batch size

3 - From our random noise data vector created in step (2) we can generate our fake image data bypassing the vector to the generator. This will be used in combination with our real data from step 1 to train the discriminator. Notice also that this time our labels vector is all zeros, since 0 represents the class label for fake images.

4 - Given the fake and real images along with their labels, we can train our discriminator for classification. The total loss will be the loss of the fake images + the loss of the real images

5 - Now that our discriminator has been updated we can use it to make predictions. The loss on those predictions will be backpropagated through the generator, such that the generator’s weights are updated specifically *according to how well it is fooling the discriminator*.  

    a) generate some fake images to make predictions on
    b) use the discriminator to make predictions on the batch of fake images and save the output.

6 - Using the predictions from the discriminator, we train our generator. Notice that we use the _real_labels_ of all 1s as the target, since the target for our generator is to create images that look real and are predicted as 1! Thus a loss of 0 for the generator would correspond the discriminator predict all 1s.

In [ ]:
# Convenience function for training our Discriminator
def train_discriminator(discriminator, real_images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()

    # Get the predictions, loss, and score of the real images
    predictions = discriminator(real_images)
    real_loss = criterion(predictions, real_labels)
    real_score = predictions

    # Get the predictions, loss, and score of the fake images
    predictions = discriminator(fake_images)
    fake_loss = criterion(predictions, fake_labels)
    fake_score = predictions

    # Calculate the total loss, update the weights, and update the optimizer
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

# Convenience function for training our Generator
def train_generator(generator, discriminator_outputs, real_labels):
    generator.zero_grad()

    # Calculate the total loss, update the weights, and update the optimizer
    g_loss = criterion(discriminator_outputs, real_labels)
    g_loss.backward()
    g_optimizer.step()
    return g_loss
    
for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):

        # (1) Prepare the real data for the Discriminator
        real_images = Variable(images) #.cuda()
        real_labels = Variable(torch.ones(images.size(0))).to(device)

        # (2) Prepare the random noise data for the Generator
        noise = Variable(torch.randn(images.size(0), 100)).to(device)

        # (3) Prepare the fake data for the Discriminator
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0))).to(device)

        # (4) Train the discriminator on real and fake data
        d_loss, real_score, fake_score = train_discriminator(discriminator,
                                                             real_images, real_labels,
                                                             fake_images, fake_labels)

        # (5a) Generate some new fake images from the Generator.
        # (5b) Get the label predictions of the Discriminator on that fake data.
        noise = Variable(torch.randn(images.size(0), 100)).to(device)
        fake_images = generator(noise)

        outputs = discriminator(fake_images)

        # (6) Train the generator
        g_loss = train_generator(generator, outputs, real_labels)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
